In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

open a database

In [ ]:
string dbPath = @"WedgeFlow_db";
var database = OpenOrCreateDatabase(dbPath);

Opening existing database 'WedgeFlow_db'.


run WedgeFlow simulation and safe to database

In [17]:
BoSSS.Solution.Application.InitMPI(num_threads:1);
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
                    dbPath:dbPath,
                    optiLSDegree: 1,
                    lsDegree: 1,
                    shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
                    MaxIterations: 200,
                    dgDegree: 0,
                    numOfCellsX: 15,
                    numOfCellsY: 10,
                    initialAngle_shockLS: 32,
                    PlotInterval: -1,
                    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
                    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
                    FluxVersion: XESF.Fluxes.FluxVersion.Optimized,
                    agg: 0.4,
                    globalization: ApplicationWithIDT.GlobalizationStrategy.LineSearch
                    );
p.Init(C);
p.RunSolverMode();


rm XESFWedgeFlow_TwoLs_0.plt XESFWedgeFlow_TwoLs_SP_0.plt XESFWedgeFlow_TwoLs_SP_1.plt XESFWedgeFlow_TwoLs_SP_10.plt XESFWedgeFlow_TwoLs_SP_11.plt XESFWedgeFlow_TwoLs_SP_12.plt XESFWedgeFlow_TwoLs_SP_13.plt XESFWedgeFlow_TwoLs_SP_14.plt XESFWedgeFlow_TwoLs_SP_15.plt XESFWedgeFlow_TwoLs_SP_16.plt XESFWedgeFlow_TwoLs_SP_17.plt XESFWedgeFlow_TwoLs_SP_18.plt XESFWedgeFlow_TwoLs_SP_19.plt XESFWedgeFlow_TwoLs_SP_2.plt XESFWedgeFlow_TwoLs_SP_20.plt XESFWedgeFlow_TwoLs_SP_21.plt XESFWedgeFlow_TwoLs_SP_22.plt XESFWedgeFlow_TwoLs_SP_23.plt XESFWedgeFlow_TwoLs_SP_24.plt XESFWedgeFlow_TwoLs_SP_25.plt XESFWedgeFlow_TwoLs_SP_26.plt XESFWedgeFlow_TwoLs_SP_27.plt XESFWedgeFlow_TwoLs_SP_28.plt XESFWedgeFlow_TwoLs_SP_29.plt XESFWedgeFlow_TwoLs_SP_3.plt XESFWedgeFlow_TwoLs_SP_30.plt XESFWedgeFlow_TwoLs_SP_31.plt XESFWedgeFlow_TwoLs_SP_32.plt XESFWedgeFlow_TwoLs_SP_33.plt XESFWedgeFlow_TwoLs_SP_34.plt XESFWedgeFlow_TwoLs_SP_35.plt XESFWedgeFlow_TwoLs_SP_36.plt XESFWedgeFlow_TwoLs_SP_37.plt XESFWedgeFlow_T

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [18]:
string dbPath = @"WedgeFlow_db";
var database = OpenOrCreateDatabase(dbPath);
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);

## Enthalpy error

enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

As we have uniform inflow the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [19]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [20]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [21]:
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M988.5,57.1 L1041.9,57.1 M53.9,45.7 L75.8,47.2 L97.7,47.6 L119.7,48.1 L141.6,53.8 L163.5,58.3
 L185.4,63.2 L207.4,63.9 L229.3,67.8 L251.2,68.6 L273.1,70.3 L295.1,74.3 L317.0,76.9 L338.9,79.8
 L360.8,81.1 L382.8,85.5 L404.7,88.0 L426.6,97.2 L448.5,115.2 L470.5,151.5 L492.4,216.3 L514.3,254.7
 L536.2,264.2 L558.2,278.1 L580.1,286.0 L602.0,289.7 L623.9,292.0 L645.8,293.0 L667.8,293.3 L689.7,293.4
 L711.6,293.5 L733.5,293.5 L755.5,293.5 L777.4,293.5 L799.3,293.5 L821.2,293.5 L843.2,293.5 L865.1,293.5
 L887.0,293.5 L908.9,293.5 L930.9,293.5 L952.8,293.5 L974.7,293.5 L996.6,293.5 L1018.6,293.5 L1040.5,293.5
 L1062.4,293.5 L1084.3,293.5 L1106.3,293.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M988.5,81.1 L1041.9,81.1 M53.9,53.1 L75.8,54.5 L97.7,54.8 L119.7,55.5 L141.6,61.8 L163.5,66.3
 L185.4,72.5 L207.4,73.1 L229.3,81.6 L251.2,82.0 L273.1,85.4 L295.1,92.0 L317.0,105.6 L338.9,96.9
 L360.8,98.2 L382.8,100.1 L404.7,101.6 L426.6,106.7 L448.5,121.5 L470.5,163.7 L492.4,230.2 L514.3,271.5
 L536.2,277.6 L558.2,292.7 L580.1,308.2 L602.0,313.1 L623.9,320.0 L645.8,329.2 L667.8,339.5 L689.7,349.4
 L711.6,359.2 L733.5,368.8 L755.5,378.2 L777.4,387.9 L799.3,396.7 L821.2,404.9 L843.2,413.3 L865.1,417.0
 L887.0,419.2 L908.9,419.8 L930.9,421.5 L952.8,424.7 L974.7,424.5 L996.6,424.5 L1018.6,424.7 L1040.5,424.8
 L1062.4,424.9 L1084.3,424.9 L1106.3,424.9 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||herr|| 
 
 
 ||h err || 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M988.5,105.1 L1041.9,105.1 M53.9,98.8 L75.8,100.3 L97.7,100.6 L119.7,101.3 L141.6,109.0 L163.5,112.2
 L185.4,119.8 L207.4,120.5 L229.3,128.2 L251.2,131.4 L273.1,157.4 L295.1,160.4 L317.0,161.8 L338.9,166.8
 L360.8,168.2 L382.8,173.0 L404.7,174.5 L426.6,182.5 L448.5,198.6 L470.5,256.6 L492.4,318.3 L514.3,338.2
 L536.2,349.5 L558.2,360.9 L580.1,365.4 L602.0,367.1 L623.9,367.7 L645.8,368.2 L667.8,368.4 L689.7,368.5
 L711.6,368.5 L733.5,368.5 L755.5,368.5 L777.4,368.5 L799.3,368.5 L821.2,368.5 L843.2,368.5 L865.1,368.5
 L887.0,368.5 L908.9,368.5 L930.9,368.5 L952.8,368.5 L974.7,368.5 L996.6,368.5 L1018.6,368.5 L1040.5,368.5
 L1062.4,368.5 L1084.3,368.5 L1106.3,368.5 '/>

In [22]:
p.PlotShadowFields(si);